# 2.1 Using Tools

## Overview

Tool calling (also known as function calling) extends basic LLM capabilities by allowing the model to execute external functions. This pattern enables:

- **External Data Access**: Retrieve real-time information (weather, stock prices, etc.)
- **Action Execution**: Perform operations beyond text generation
- **Structured Outputs**: Get formatted responses through function parameters

### Key Concept

**Important**: The LLM doesn't actually execute tools itself. Instead:
1. The LLM **analyzes the request** and determines which tool to use
2. The LLM **returns structured instructions** about what tool to call
3. **Your application** executes the tool and gets the result
4. **Your application** sends the tool result back to the LLM
5. The LLM **formulates a final response** using the tool output

This requires **two separate LLM calls**: one to determine the tool, and another to process the result.

## Implementation Steps

In [1]:
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, ToolCall, ToolMessage
from typing import Literal
from langchain.tools import tool

### Step 1: Initialize the Language Model

Same as before, we start with our base LLM connection.

In [2]:
llm = AzureChatOpenAI(model="gpt-4.1-mini")

### Step 2: Define Tools/Functions

We create a tool using the `@tool` decorator. This tool:
- **Has a clear docstring** describing its purpose and parameters
- **Uses type hints** to specify allowed input values
- **Returns structured data** that the LLM can interpret

The `Literal` type restricts inputs to specific cities, making the tool more reliable.

In [3]:
@tool
def weather_tool(location: Literal["Chicago", "New York", "Los Angeles"]) -> str:
    """
    Get the current weather for a specified location.
    Args:
        location (str): The name of the city to get the weather for. 
                        Must be one of "Chicago", "New York", or "Los Angeles".
    Returns:
        str: A string describing the current weather in the specified location.
    """

    weather_data = {
        "New York": "Sunny, 25°C",
        "Los Angeles": "Cloudy, 22°C",
        "Chicago": "Rainy, 18°C"
    }
    return weather_data.get(location, "Weather data not available for this location.")

### Step 3: Bind Tools to the LLM

`bind_tools()` tells the LLM about available functions. The LLM now knows:
- What tools are available
- What parameters each tool expects  
- When each tool should be used

This creates a new LLM instance that's "tool-aware".

In [4]:
llm_with_tools = llm.bind_tools([weather_tool])

### Step 4: Create the Initial Conversation

We set up a conversation asking about weather information. Notice the user asks about "NYC" - the LLM will need to map this to "New York" for our tool.

In [5]:
messages = [
    SystemMessage(
        content="You are a helpful assistant that can provide weather information for specific cities."
    ),
    HumanMessage(
        content="What is the weather like in NYC?"
    )
]

### Step 5: First LLM Call - Tool Selection

The LLM analyzes the request and determines that weather information is needed. Instead of generating a text response, it returns:
- **Tool call instructions** with the function name and parameters
- **A unique call ID** to track this specific tool invocation

We add this response to our message history to maintain context.

In [6]:
response = llm_with_tools.invoke(input=messages)
#Adding the response to the prompt history
messages.append(response)

### Step 6: Execute Tools and Second LLM Call

Here's where the magic happens:

1. **Check for tool calls**: If the LLM requested tools, we process them
2. **Execute each tool**: We run the actual function with the LLM's parameters
3. **Create tool messages**: Package the results in ToolMessage objects
4. **Add to conversation**: Include tool results in the message history
5. **Second LLM call**: The LLM formulates a final response using the tool output

The `tool_call_id` links the tool result back to the original request.

In [7]:
if len(response.tool_calls) > 0:
    for tool_call in response.tool_calls:
        messages.append(
            # Adding the tool call to the prompt history
            ToolMessage(
                name=tool_call["name"],
                content=weather_tool.run(tool_call["args"]["location"]), #Executing the tool
                tool_call_id=tool_call["id"],
            )
        )

    # Calling the LLM again with the updated messages, containing the tool response
    response = llm_with_tools.invoke(input=messages)
    # Adding the final response to the prompt history
    messages.append(response)

### Step 7: View the Complete Conversation Flow

The output shows the complete conversation including:
- Original system and human messages
- AI message with tool call instructions
- Tool message with the actual result
- Final AI message with the formatted response

In [8]:
for msg in messages:
    msg.pretty_print()

================================ System Message ================================

You are a helpful assistant that can provide weather information for specific cities.
================================ Human Message =================================

What is the weather like in NYC?
================================== Ai Message ==================================
Tool Calls:
  weather_tool (call_4d03xCDEnFotRMjeg8w5UP4j)
 Call ID: call_4d03xCDEnFotRMjeg8w5UP4j
  Args:
    location: New York
================================= Tool Message =================================
Name: weather_tool

Sunny, 25°C
================================== Ai Message ==================================

The weather in New York City is currently sunny with a temperature of 25°C.


## Understanding the Tool Call Flow

Notice the Tool Calls and Tool Messages in the conversation flow above. This demonstrates the complete cycle:

1. **Human Message**: "What is the weather like in NYC?"
2. **AI Message with Tool Call**: LLM decides to use `weather_tool` with parameter `location: "New York"`
3. **Tool Message**: Actual execution result: "Sunny, 25°C"
4. **Final AI Message**: Formatted response incorporating the tool output

**Important**: Tool usage means **2 LLM calls** minimum:
- **First call**: Determine what tool is necessary
- **Second call**: Generate response containing the tool result

## Key Takeaways

1. **Tool Binding**: Use `bind_tools()` to make functions available to the LLM
2. **Tool Detection**: Check `response.tool_calls` to see if tools are needed
3. **Tool Execution**: Your application executes tools, not the LLM
4. **Result Integration**: Tool outputs become part of the conversation context
5. **Cost Consideration**: Each tool use requires multiple LLM calls

## When to Use Tool Calling

- **Real-time Data**: Weather, stock prices, current events
- **External APIs**: Database queries, web searches, social media
- **Calculations**: Math operations, data processing
- **File Operations**: Reading, writing, or analyzing files
- **System Actions**: Sending emails, creating calendar events

## Next Steps

This manual tool calling approach works for simple cases, but becomes complex with:
- Multiple tools
- Conditional tool sequences
- Error handling and retries

The next notebook will show **ReAct Agents** that automate this tool calling process.